# Try one

## import libs and define vars

In [6]:
import pkg_resources
import sys
import numpy as np
import h5py
from scipy import stats
import time
import sklearn
from sklearn.linear_model import LinearRegression 
import math as math
#import statsmodels.api as sm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import wbplot
from wbplot import pscalar
import ActflowToolbox as actflow
import ActflowToolbox.connectivity_estimation as fc


## Load data

In [15]:
actflow_dir = pkg_resources.resource_filename("ActflowToolbox.examples", "HCP_example_data/" )
subj_nums = ["100206", "108020", "117930", "126325", "133928", "143224", "153934", "164636", "174437", "183034", "194443", "204521", "212823", "268749", "322224", "385450", "463040", "529953", "587664", "656253", "731140", "814548", "877269", "978578", "100408", "108222", "118124", "126426", "134021", "144832"]
n_trs = 1195
# Load task activations (GLM betas); 360 MMP regions x 24 HCP tasks x 30 HCP subjects 
file_path = actflow_dir + "HCP_example_taskactivations_data" + ".h5"
h5f = h5py.File(file_path, "r")
activity_data = h5f["taskbeta"][:]
h5f.close()
n_nodes, n_conditions, n_subjs = activity_data.shape
# Load resting-state fMRI data: 360 MMP regions x one run of resting-state fMRI (1195 time points) x 30 HCP subjects
rest_data = np.zeros((n_nodes, n_trs, n_subjs))
for subj_ix in range(n_subjs):
    file_path = (actflow_dir + "HCP_example_restrun1_subj" + subj_nums[subj_ix] + "_data" + ".h5")
    h5f = h5py.File(file_path, "r")
    rest_data[:, :, subj_ix] = h5f["restdata"][:] 
    h5f.close()

## Activity flow mapping with regularized partial correlation

In [26]:
# Exclude source vertices 10 mm from target nodes to avoid circularity
# Using preloaded vertex-wise data: activity_data_vertex and rest_data_vertex
# Non-circular task activations
activity_data_noncirc = np.zeros((n_nodes, n_conditions, n_subjs)) 
for subj_ix in range(n_subjs):
    activity_data_here = activity_data[:, :, subj_ix] 
    activity_data_noncirc_all = fc.calcactivity_parcelwise_noncircular(activity_data)
    for node_ix in range(n_nodes):
        for cond_ix in range(n_conditions):
            extracted_data = activity_data_noncirc_all[node_ix, node_ix, cond_ix] 
            activity_data_noncirc[node_ix, cond_ix, subj_ix] = extracted_data.copy()
# Non-circular resting-state FC
fc_arr_noncirc = np.zeros((n_nodes, n_nodes, n_subjs)) 
for subj_ix in range(n_subjs):
    rest_data_here = rest_data[:, :, subj_ix]
# Can specify connmethod = "combinedFC" here
    fc_arr_noncirc[:, :, subj_ix] = fc.calcconn_parcelwise_noncircular(rest_data)

ExpiredDeprecationError: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Raises <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0

In [39]:
file="/Users/gabrieledele/miniforge3/envs/mlproj/lib/python3.9/site-packages/ActflowToolbox/examples/HCP_example_data/HCP_example_taskactivations_data.h5"
with h5py.File(file_path, 'r') as hf:
    # List all datasets in the file
    print("Datasets in HDF5 file:")
    for dataset_name in hf.keys():
        print(dataset_name)
        data = hf[dataset_name][:]
        print(type(data))
np.savetxt('/Users/gabrieledele/Desktop/dataH5.csv', data, delimiter=',')

Datasets in HDF5 file:
restdata
<class 'numpy.ndarray'>
